In [3]:
#data analysis for the FRED API

In [4]:
pip install requests pandas



[notice] A new release of pip is available: 23.2.1 -> 25.0.1
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [4]:
import requests
import time
import pandas as pd 

In [2]:
api_key = "b22616cbd7e8b1c6a98f6f5c9669421f"

In [17]:
#searching for relevant ids
base_url = f"https://api.stlouisfed.org/fred/series/search"
search_item = "unemployment"
params = {
    "search_text": search_item,
    "api_key": api_key,
    "file_type": "json"
}

response = requests.get(base_url, params=params)
data = response.json()

In [18]:
for i, series in enumerate(data['seriess'][:20]):
    print(f"{i+1}. {series['title']}")
    print(f"   ID: {series['id']}")
    print(f"   Frequency: {series['frequency']}")
    print(f"   Units: {series['units']}")
    print()

1. Unemployment Rate
   ID: UNRATE
   Frequency: Monthly
   Units: Percent

2. Unemployment Rate
   ID: UNRATENSA
   Frequency: Monthly
   Units: Percent

3. Unemployment Level
   ID: UNEMPLOY
   Frequency: Monthly
   Units: Thousands of Persons

4. Noncyclical Rate of Unemployment
   ID: NROU
   Frequency: Quarterly
   Units: Percent

5. Continued Claims (Insured Unemployment)
   ID: CCSA
   Frequency: Weekly, Ending Saturday
   Units: Number

6. Continued Claims (Insured Unemployment)
   ID: CCNSA
   Frequency: Weekly, Ending Saturday
   Units: Number

7. Unemployment Rate - Black or African American
   ID: LNS14000006
   Frequency: Monthly
   Units: Percent

8. Unemployment Level
   ID: LNU03000000
   Frequency: Monthly
   Units: Thousands of Persons

9. Unemployment Rate - Black or African American
   ID: LNU04000006
   Frequency: Monthly
   Units: Percent

10. Total Unemployed, Plus All Persons Marginally Attached to the Labor Force, Plus Total Employed Part Time for Economic Reas

In [16]:
#collect data 

base_url_meta_data = f"https://api.stlouisfed.org/fred/series"

series_id_list = [
    "GDP",
    "GDPC1",
    "GDPA",
    "PCEC",
    "PCECA",
    "A091RC1Q027SBEA",
    "PSAVE",
    "A071RC1A027NBEA",
    "A939RX0Q048SBEA",
    "A229RX0"
]

collected_meta_data = []

for id in series_id_list:
    params_meta_data = {
        "api_key": api_key,
        "file_type": "json",
        "series_id": id,
        "realtime_start": "1980-01-01"
    }
    
    response = requests.get(base_url_meta_data, params=params_meta_data)
    if response.status_code >= 400:
        print(f"Error {response.status_code}, at iteration series id: {id}")
        break
    else:
        print("appending json data to collected_data")
        data = response.json()
        collected_meta_data.append(data)

appending json data to collected_data
appending json data to collected_data
appending json data to collected_data
appending json data to collected_data
appending json data to collected_data
appending json data to collected_data
appending json data to collected_data
appending json data to collected_data
appending json data to collected_data
appending json data to collected_data


In [15]:
meta_data_df = pd.DataFrame(collected_meta_data)

meta_data_df["seriess"]

0    [{'id': 'GDP', 'realtime_start': '1991-12-04',...
1    [{'id': 'GDPC1', 'realtime_start': '1991-12-04...
2    [{'id': 'GDPA', 'realtime_start': '1991-12-04'...
3    [{'id': 'PCEC', 'realtime_start': '1980-01-01'...
4    [{'id': 'PCECA', 'realtime_start': '1980-01-01...
5    [{'id': 'A091RC1Q027SBEA', 'realtime_start': '...
6    [{'id': 'PSAVE', 'realtime_start': '1980-01-01...
7    [{'id': 'A071RC1A027NBEA', 'realtime_start': '...
8    [{'id': 'A939RX0Q048SBEA', 'realtime_start': '...
9    [{'id': 'A229RX0', 'realtime_start': '2010-05-...
Name: seriess, dtype: object

In [20]:
base_url_data = f"https://api.stlouisfed.org/fred/series/observations"

collected_data = {}

for id in series_id_list:
    params_data = {
        "api_key": api_key,
        "file_type": "json",
        "series_id": id,
        "limit": 100000,
        "units": "lin"
    }
    
    response = requests.get(base_url_data, params=params_data)
    if response.status_code >= 400:
        print(f"Error {response.status_code}, at iteration series id: {id}")
        break
    else:
        print("appending data to collected_data")
        data = response.json()
        observations = data.get("observations", [])
    
        dates = [obs["date"] for obs in observations]
        values = [float(obs["value"]) if obs["value"] != "." else None for obs in observations]
    
        series = pd.Series(data=values, index=pd.to_datetime(dates), name=id)
        collected_data[id] = series

appending data to collected_data
appending data to collected_data
appending data to collected_data
appending data to collected_data
appending data to collected_data
appending data to collected_data
appending data to collected_data
appending data to collected_data
appending data to collected_data
appending data to collected_data


In [23]:
for id in series_id_list:
    print(collected_data[id].head())

1946-01-01        NaN
1946-04-01        NaN
1946-07-01        NaN
1946-10-01        NaN
1947-01-01    243.164
Name: GDP, dtype: float64
1947-01-01    2182.681
1947-04-01    2176.892
1947-07-01    2172.432
1947-10-01    2206.452
1948-01-01    2239.682
Name: GDPC1, dtype: float64
1929-01-01    104.556
1930-01-01     92.160
1931-01-01     77.391
1932-01-01     59.522
1933-01-01     57.154
Name: GDPA, dtype: float64
1939-01-01   NaN
1939-04-01   NaN
1939-07-01   NaN
1939-10-01   NaN
1940-01-01   NaN
Name: PCEC, dtype: float64
1929-01-01    77.382
1930-01-01    70.136
1931-01-01    60.672
1932-01-01    48.714
1933-01-01    45.945
Name: PCECA, dtype: float64
1947-01-01    5.352
1947-04-01    5.360
1947-07-01    5.356
1947-10-01    5.440
1948-01-01    5.476
Name: A091RC1Q027SBEA, dtype: float64
1939-01-01   NaN
1939-04-01   NaN
1939-07-01   NaN
1939-10-01   NaN
1940-01-01   NaN
Name: PSAVE, dtype: float64
1929-01-01    3.967
1930-01-01    3.344
1931-01-01    2.822
1932-01-01   -0.088
1933-01-

In [24]:
print(collected_data.keys())

dict_keys(['GDP', 'GDPC1', 'GDPA', 'PCEC', 'PCECA', 'A091RC1Q027SBEA', 'PSAVE', 'A071RC1A027NBEA', 'A939RX0Q048SBEA', 'A229RX0'])


In [25]:
for id in series_id_list:
    print(collected_data[id].info())

<class 'pandas.core.series.Series'>
DatetimeIndex: 316 entries, 1946-01-01 to 2024-10-01
Series name: GDP
Non-Null Count  Dtype  
--------------  -----  
312 non-null    float64
dtypes: float64(1)
memory usage: 4.9 KB
None
<class 'pandas.core.series.Series'>
DatetimeIndex: 312 entries, 1947-01-01 to 2024-10-01
Series name: GDPC1
Non-Null Count  Dtype  
--------------  -----  
312 non-null    float64
dtypes: float64(1)
memory usage: 4.9 KB
None
<class 'pandas.core.series.Series'>
DatetimeIndex: 96 entries, 1929-01-01 to 2024-01-01
Series name: GDPA
Non-Null Count  Dtype  
--------------  -----  
96 non-null     float64
dtypes: float64(1)
memory usage: 1.5 KB
None
<class 'pandas.core.series.Series'>
DatetimeIndex: 344 entries, 1939-01-01 to 2024-10-01
Series name: PCEC
Non-Null Count  Dtype  
--------------  -----  
312 non-null    float64
dtypes: float64(1)
memory usage: 5.4 KB
None
<class 'pandas.core.series.Series'>
DatetimeIndex: 96 entries, 1929-01-01 to 2024-01-01
Series name: PCEC

In [27]:
import pickle

with open("collected_data.pkl", "wb") as f:
    pickle.dump(collected_data, f)